# Target Percent Pipeline Algorithm

利用pipeline提供的買賣清單與持股權重進行定期再平衡的演算法。

<span id="menu"></span>
本文件包含以下部份：

**基本設定**
1. [Set Environment Variables](#Env)
2. [Investment Universe](#Universe)
3. [Ingest](#Ingest)
4. [Imports](#Imports)
5. [Pipeline](#Pipeline)

**參數說明與範例**

6. [API Reference](#APIReference)：參數說明。
   - [class zipline.algo.pipeline_algo.TargetPercentPipeAlgo](#APIReference)
   - [class zipline.api.date_rules](#date_rules)


7. [Examples](#Examples)：範例。
   - [Case 1－調整start_session與end_session](#Case1)
   - [Case 2－調整max_leverage](#Case2)
   - [Case 3－調整tradeday](#Case3)
   - [Case 4－調整rebalance_date_rule](#Case4)
   - [Case 5－調整slippage_model](#Case5)
   - [Case 6－調整stocklist](#Case6)
   - [Case 7－調整order_filling_policy](#Case7)
   - [Case 8－調整allow_short](#Case8)
   - [Case 9－調整cancel_datedelta](#Case9)   
   - [Case 10－調整limit_buy_multiplier](#Case10)  
   - [Case 11－調整custom_weight、analyze、record_vars、get_record_vars與get_transaction_detail](#Case11)   

<span id="Env"></span>
# 1. Set Environment Variables
[Return to Menu](#menu)

In [1]:
import pandas as pd
import datetime
import tejapi
import os

# set tej_key and base
os.environ['TEJAPI_KEY'] = "your key" 
os.environ['TEJAPI_BASE'] = "https://api.tej.com.tw"

# set benchmark
benchmark=['IR0001']

# set calendar
calendar_name='TEJ_XTAI'

# set bundle name
bundle_name = 'tquant'

# set date
start='2023-06-01'
end='2023-10-03'

# 由文字型態轉為Timestamp，供回測使用
tz = 'UTC'
start_dt, end_dt = pd.Timestamp(start, tz = tz), pd.Timestamp(end, tz = tz)

# 設定os.environ['mdate'] = start+' '+end，供ingest bundle使用
os.environ['mdate'] = start+' '+end

<span id="Universe"></span>
# 2. Investment Universe

台灣50指數成分股

[Return to Menu](#menu)

In [2]:
from zipline.sources.TEJ_Api_Data import get_universe

StockList = get_universe(start, end, idx_id='IX0002')

print(len(StockList))

Currently used TEJ API key call quota 67/9223372036854775807 (0.0%)
Currently used TEJ API key data quota 2255014/9223372036854775807 (0.0%)
55


In [3]:
os.environ['ticker'] = ' '.join(StockList + benchmark)

In [4]:
os.environ['ticker']

'1101 1216 1301 1303 1326 1402 1590 1605 2002 2207 2301 2303 2308 2317 2327 2330 2345 2357 2379 2382 2395 2408 2412 2454 2603 2609 2615 2801 2880 2881 2882 2883 2884 2885 2886 2887 2890 2891 2892 2912 3008 3034 3037 3045 3231 3711 4904 4938 5871 5876 5880 6415 6505 6669 9910 IR0001'

<span id="Ingest"></span>
# 3. Ingest
[Return to Menu](#menu)

In [5]:
# Ingest pricing bundle
!zipline ingest -b tquant

Merging daily equity files:
Currently used TEJ API key call quota 72/9223372036854775807 (0.0%)
Currently used TEJ API key data quota 2261479/9223372036854775807 (0.0%)


[2024-03-13 02:40:44.060855] INFO: zipline.data.bundles.core: Ingesting tquant.
[2024-03-13 02:40:49.764291] INFO: zipline.data.bundles.core: Ingest tquant successfully.


In [6]:
from zipline.data.data_portal import get_bundle_price

df_bundle_price = get_bundle_price(bundle_name=bundle_name,
                                   calendar_name=calendar_name,
                                   start_dt=start_dt,
                                   end_dt=end_dt)

<span id="Imports"></span>
# 4. Imports & Settings
[Return to Menu](#menu)

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
from time import time
import numpy as np
import pandas as pd
import empyrical as ep

from logbook import Logger, StderrHandler, INFO, WARNING
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator


from TejToolAPI.TejToolAPI import *

from zipline.api import record

from zipline.utils.calendar_utils import get_calendar
from zipline.utils.run_algo import  (get_transaction_detail,
                                     get_record_vars)

from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.filters import SingleAsset
from zipline.pipeline.factors import RSI
from zipline.pipeline.data import TQAltDataSet

from zipline.finance import slippage

from zipline.TQresearch.tej_pipeline import run_pipeline

# 設定log顯示方式
log_handler = StderrHandler(format_string='[{record.time:%Y-%m-%d %H:%M:%S.%f}]: ' +
                            '{record.level_name}: {record.func_name}: {record.message}',
                            level=INFO)
log_handler.push_application()
log = Logger('Algorithm')

In [9]:
import warnings
warnings.filterwarnings('ignore')

<span id="Pipeline"></span>
# 5. Pipeline

取得`Market_Cap_Dollars`（市值）資料

[Return to Menu](#menu)

In [10]:
col = ['Market_Cap_Dollars']

fields = ''
for i in col:
    fields += i
    fields += ' '

os.environ['fields'] = fields

In [11]:
# Ingest Fundamental Bundle
!zipline ingest -b fundamentals

Currently used TEJ API key call quota 83/9223372036854775807 (0.0%)
Currently used TEJ API key data quota 2298569/9223372036854775807 (0.0%)


[2024-03-13 02:40:54.272410] INFO: zipline.data.bundles.core: Ingesting fundamentals.
[2024-03-13 02:41:05.038363] INFO: zipline.data.bundles.core: Ingest fundamentals successfully.


<span id="APIReference"></span>
# 6. API Reference
[Return to Menu](#menu)

### **class zipline.algo.pipeline_algo.<font color=DeepPink>TargetPercentPipeAlgo</font>** 
*(self, bundle_name='tquant', start_session=None, end_session=None, trading_calendar=get_calendar('TEJ_XTAI'),*

*capital_base=1e7, data_frequency='daily', tradeday=None, rebalance_date_rule=None, stocklist=None, benchmark='IR0001',* 

*zero_treasury_returns=True, slippage_model=slippage.VolumeShareSlippage(volume_limit=0.025, price_impact=0.1),*

*commission_model=commission.Custom_TW_Commission(min_trade_cost=20, discount = 1.0, tax = 0.003), max_leverage=0.8,* 

*limit_buy_multiplier=None, limit_sell_multiplier=None, allow_short=False, cancel_datedelta=None, custom_weight=False,*

*custom_loader=None, pipeline=None, analyze=None, record_vars=None, get_record_vars=False, get_transaction_detail=False,*

*order_filling_policy='next_bar')*

利用pipeline提供的買賣清單與持股權重進行定期再平衡的演算法。必要參數僅有`pipeline`。


#### Parameters:

- **bundle_name** (*str, optional*)－bundle名稱。預設是 **`'tquant'`**。  


- **start_session** (*pd.Timestamp or datetime, optional*)－回測起始日期。預設是**bundle中最早的資料日期**。  


- **end_session** (*pd.Timestamp or datetime, optional*)－回測結束日期。預設是**bundle中最晚的資料日期**。  


- **trading_calendar** (*TradingCalendar, optional*)－
  - 設置交易日曆。預設是 **`get_calendar('TEJ_XTAI')`**。
  - TradingCalendar：`zipline.utils.calendar_utils.TradingCalendar`。  


- **capital_base** (*float, optional*)－初始資金額度。預設是**一千萬**。  


- **data_frequency** (*{'daily', 'minute'}, optional*)－資料頻率，目前僅支援日頻率 **`'daily'`**。  


- **tradeday** (*list[str] or list[pd.Timestamp] or pd.DatetimeIndex, optional*)－
  - 交易日期清單，限制只能在這個清單中的日期進行交易。預設是**None**，代表**每日**都交易。
  - `rebalance_date_rule`與`tradeday`請擇一設定，若兩者皆設定，則會以`rebalance_date_rule`為主。  


- **rebalance_date_rule**(*EventRule, optional*)－
  - 交易頻率，設定固定的交易頻率。預設是**None**，代表**每日**都交易。
  - `rebalance_date_rule`與`tradeday`請擇一設定，若兩者皆設定，則會以`rebalance_date_rule`為主。
  - EventRule：`zipline.utils.events.EventRule`。可使用的`rebalance_date_rule`參考[zipline.api.date_rules](#date_rules)。  


- **stocklist** (*list[str], optional*)－交易清單，限制只能交易這個清單中的股票。預設是**None**，代表使用所有bundle中的股票。  


- **benchmark** (*str, optional*)－指數名稱，用來與投資組合報酬率比較。預設是`'IR0001'`，代表**台灣發行量加權股價報酬指數**。  


- **zero_treasury_returns** (*bool, optional*)－
  - 是否將無風險利率設定為0，預設是**True**，代表設定為0。因此使用`algo.run()`產出的回測報表中，`treasury_return`與`treasury_period_return`皆會是0。
  - 若設定為**False**，則會<font color=DeepPink>耗費額外API流量</font>，並取得第一銀行一年期定存利率作為無風險利率。  


- **slippage_model** (*SlippageModel, optional*)－
  - 設定滑價模型。預設為`slippage.VolumeShareSlippage(volume_limit=0.025, price_impact=0.1)`。可使用的模型參考：[lecture/Zipline Slippage.ipynb](https://github.com/tejtw/TQuant-Lab/blob/main/lecture/Zipline%20Slippage.ipynb)
  - SlippageModel：`zipline.finance.slippage.SlippageModel`。


- **commission_model** (*CommissionModel, optional*)－
  - 設定手續費模型。預設為`commission.Custom_TW_Commission(min_trade_cost=20, discount=1.0, tax = 0.003)`。可使用的模型參考：[lecture/Zipline Commission Models.ipynb](https://github.com/tejtw/TQuant-Lab/blob/main/lecture/Zipline%20Commission%20Models.ipynb)
  - CommissionModel：`zipline.finance.commission.CommissionModel`


- **max_leverage** (*float, optional*)－槓桿限制，預設 = **0.8**。  


- **limit_buy_multiplier** (*float, optional*)－
  - 買進／回補時的limit_price乘數，若有提供則limit_price = 下單時最近一筆收盤價 * `limit_buy_multiplier`。
  - 預設為**None**，代表**不設定**買進／回補時的limit_price。  


- **limit_sell_multiplier** (*float, optional*)－
  - 賣出／放空時的limit_price乘數，若有提供則limit_price = 下單時最近一筆收盤價 * `limit_sell_multiplier`。
  - 預設為**None**，代表**不設定**賣出／放空時的limit_price。  


- **allow_short** (*bool, optional*)－是否允許放空股票，預設為**False**，代表僅能做多。若設定為**True**，則pipeline中需要有`shorts`欄位。  


- **cancel_datedelta** (*int, optional*)－訂單幾天內未完全成交就取消。預設是在**下一次再平衡**時取消。 


- **custom_weight** (*bool, optional*)－
  - 是否要使用自訂的加權權數，預設為**False**，代表不使用（即等權重加權）。
  - 若設定為**True**，則pipeline中需要有`long_weights`（若`allow_short`=True，則也須有`short_weights`）欄位。


- **custom_loader** (*PipelineLoader , optional*)－
  - 用來取得價量以外資料，預設是**None**，代表不使用價量、`TQDataSet`與`TQAltDataSet`以外資料。
  - TQDataSet：`zipline.pipeline.data.TQDataSet`
  - TQAltDataSet：`zipline.pipeline.data.TQAltDataSet`
  - 目前支援的`PipelineLoader`：
    - DataFrameLoader（`zipline.pipeline.loaders.frame.DataFrameLoader`）。


- **pipeline** (*Pipeline*)－
  - 要用來產出交易清單或權重的pipeline，為<font color=DeepPink>**必要參數**</font>。
  - Pipeline：`zipline.pipeline.Pipeline`


- **analyze** (*callable[(context, pd.DataFrame) -> None], optional*)－
  - 傳入`analyze`函式以用於回測，函式中必須要有`context`與`perf`參數，預設是**None**。
  - 此函式在**回測結束**時被一次性呼叫，並繪製自訂圖表。


- **record_vars** (*callable[(context, BarData) -> None], optional*)－
  - 傳入`record_vars`函式以用於回測，函式中必須要有`context`與`data`參數，預設是**None**。
  - 此函式在**每個交易日結束**時被呼叫，並把指定資料紀錄於回測結果的DataFrame中。


- **get_record_vars** (*bool, optional*)－
  - 是否產出`record_vars`中`record`方法所記錄的變數，預設為**False**，代表不產出。
  - 若設定為**True**，則可用`algo.dict_record_vars`取出。


- **get_transaction_detail** (*bool, optional*)－
  - 是否產出交易結果，預設為**False**，代表不產出。
  - 若設定為**True**，則可用`algo.positions`、`algo.transactions`、`algo.orders`方式取出交易結果。


- **order_filling_policy** (*{'next_bar','current_bar'}, optional*)－
  - 設定交易方式，預設為**next_bar**，代表當天收盤後下單，隔一日收盤前成交，也就是原先的回測方式。
  - 若要當天開盤前下單，收盤前成交，則需指定設定為**current_bar**。

  
#### Returns:
    algo

#### Return type:
    zipline.algo.pipeline_algo.TargetPercentPipeAlgo


<br>




<br/>

**<font color=DeepPink>run</font>()**
>
> 執行演算法
> 
> **Returns:**
> - perf（回測報表）
> 
> **Return type:**
> - pd.DataFrame

<span id="date_rules"></span>
### **class zipline.api.<font color=DeepPink>date_rules</font>**
[Return to Menu](#menu)

- 為Factory API，主要用來傳入`zipline.api.schedule_function`的`date_rule`參數中。用來決定要以何種頻率觸發某項規則。
- 使用前請先import：`from zipline.api import date_rules`。
---

*static* **<font color=DeepPink>every_day</font>**()
> 
> 每日觸發某項規則。
> 
> **Returns:**
> - rule
> 
> **Return type:**
> - zipline.utils.events.EventRule

<br>

*static* **<font color=DeepPink>month_end</font>**(days_offset=0)
> 
> 每個月底觸發某項規則，並可以選擇性的新增一個偏移量。
> 
> **Parameters:**
> - **days_offset**(*int , optional*)：
>   - 在月底之前的第幾個交易日（由0開始計算）觸發某項規則。預設值是0，即在月底的最後一個交易日觸發。
>   - days_offset只能介於0~22之間。 
> 
> **Returns:**
> - rule
> 
> **Return type:**
> - zipline.utils.events.EventRule


<br>

*static* **<font color=DeepPink>month_start</font>**(days_offset=0)
> 
> 每個月初觸發某項規則，並可以選擇性的新增一個偏移量。
> 
> **Parameters:**
> - **days_offset**(*int , optional*)：
>   - 在月初之後的第幾個交易日（由0開始計算）才觸發某項規則。預設值是0，即在月初的第一個交易日觸發。
>   - days_offset只能介於0~22之間。 
> 
> **Returns:**
> - rule
> 
> **Return type:**
> - zipline.utils.events.EventRule


<br>

*static* **<font color=DeepPink>week_end</font>**(days_offset=0)
> 
> 在每周最後一個交易日觸發某項規則，並可以選擇性的新增一個偏移量。
> 
> **Parameters:**
> - **days_offset**(*int , optional*)：
>   - 在每周倒數第幾個交易日（由0開始計算）觸發某項規則。預設值是0，即在每周的最後一個交易日觸發。
>   - days_offset只能介於0~4之間。 
>
> **Returns:**
> - rule
> 
> **Return type:**
> - zipline.utils.events.EventRule


<br>

*static* **<font color=DeepPink>week_start</font>**(days_offset=0)
> 
> 在每周的第一個交易日觸發某項規則，並可以選擇性的新增一個偏移量。
> 
> **Parameters:**
> - **days_offset**(*int , optional*)：
>   - 在每周的第幾個交易日（由0開始計算）才觸發某項規則。預設值是0，即在每周的第一個交易日觸發。
>   - days_offset只能介於0~4之間。
> 
> **Returns:**
> - rule
> 
> **Return type:**
> - zipline.utils.events.EventRule

<span id="Examples"></span>
# 7. Examples
[Return to Menu](#menu)

In [12]:
from zipline.utils.algo_instance import get_algo_instance, set_algo_instance
from zipline.algo.pipeline_algo import TargetPercentPipeAlgo

<span id="Case1"></span>
## Case 1 調整start_session與end_session

[Return to Menu](#menu)

僅調整`start_session`與`end_session`。其餘保持預設值。

以下設定pipeline（`make_pipeline()`），並定義`longs`欄位用來判斷須持有的股票。在`longs`欄位中要持有的股票標記為True，反之標記為False。

In [13]:
from zipline.data import bundles

bundle = bundles.load(bundle_name)

def make_pipeline():
    rsi = RSI()
    longs = rsi.top(2, mask = ~SingleAsset(bundle.asset_finder.lookup_symbol('IR0001', as_of_date=None)))

    return Pipeline(
        
        columns = {
            "longs" : longs,
        }
    )

In [14]:
algo_start = '2023-09-21'
algo_start_dt = pd.Timestamp(algo_start, tz = tz)

result = run_pipeline(make_pipeline(), algo_start, end)
result.query('longs == True')

longs
2023-09-22 00:00:00+00:00 Equity(6 [1590])    True
                          Equity(24 [2603])   True
2023-09-25 00:00:00+00:00 Equity(25 [2609])   True
                          Equity(34 [2886])   True
2023-09-26 00:00:00+00:00 Equity(33 [2885])   True
                          Equity(34 [2886])   True
2023-09-27 00:00:00+00:00 Equity(25 [2609])   True
                          Equity(34 [2886])   True
2023-09-28 00:00:00+00:00 Equity(6 [1590])    True
                          Equity(34 [2886])   True
2023-10-02 00:00:00+00:00 Equity(24 [2603])   True
                          Equity(34 [2886])   True
2023-10-03 00:00:00+00:00 Equity(6 [1590])    True
                          Equity(14 [2327])   True

### 執行演算法
1. 實體化`TargetPercentPipeAlgo`並命名為`algo`。
2. 設定演算法：`set_algo_instance(algo)`
3. 執行演算法，並產出回測報表`stats`（*pd.DataFrame*）：`stats = algo.run()`

In [15]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     pipeline=make_pipeline,
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2024-03-13 02:41:05.491033]: INFO: rebalance: Cancel_order: current time: 2023-09-22 , created: 2023-09-21 , asset: Equity(46 [4904]), amount: 55478 , filled: 51900
[2024-03-13 02:41:05.499675]: INFO: rebalance: Cancel_order: current time: 2023-09-25 , created: 2023-09-25 , asset: Equity(46 [4904]), amount: -14450 , filled: 0
[2024-03-13 02:41:05.504262]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54616.31935744
[2024-03-13 02:41:05.505924]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4062, cost_basis: 982.73, last_sale_price: 981.0
[2024-03-13 02:41:05.506228]: INFO: handle_split: returning cash: 643.94
[2024-03-13 02:41:05.549641]: INFO: handle_simulation_end: Simulated 8 trading days
first open: 2023-09-21 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


查看演算法中的參數設定

In [16]:
algo

TargetPercentPipeAlgo(
    sim_params = 
SimulationParameters(
    start_session=2023-09-21 00:00:00+00:00,
    end_session=2023-10-03 00:00:00+00:00,
    capital_base=10000000.0,
    data_frequency=daily,
    emission_rate=daily,
    first_open=2023-09-21 01:01:00+00:00,
    last_close=2023-10-03 05:30:00+00:00,
    trading_calendar=<exchange_calendars.exchange_calendar_tejxtai.TEJ_XTAIExchangeCalendar object at 0x0000017B415030D0>
),
    benchmark = IR0001,
    zero treasury returns or not（if "True" then treasury returns = 0）= True,
    max_leverage = 0.8,
    slippage model used = VolumeShareSlippage(
    volume_limit=0.025,
    price_impact=0.1),
    commission_model = Custom_TW_Commission(min_trade_cost=20.0),
    liquidity_risk_management_rule = None,
    order_filling_policy = next_bar,
    adjust amount or not = False,
    limit_buy_multiplier = None,
    limit_sell_multiplier = None,
    allow short or not（if "False" then long only）= False,
    use custom weight or not（if not 

In [17]:
stats.T

,2023-09-21 13:30:00+08:00,2023-09-22 13:30:00+08:00,2023-09-25 13:30:00+08:00,2023-09-26 13:30:00+08:00,2023-09-27 13:30:00+08:00,2023-09-28 13:30:00+08:00,2023-10-02 13:30:00+08:00,2023-10-03 13:30:00+08:00
period_open,2023-09-21 09:01:00+08:00,2023-09-22 09:01:00+08:00,2023-09-25 09:01:00+08:00,2023-09-26 09:01:00+08:00,2023-09-27 09:01:00+08:00,2023-09-28 09:01:00+08:00,2023-10-02 09:01:00+08:00,2023-10-03 09:01:00+08:00
period_close,2023-09-21 13:30:00+08:00,2023-09-22 13:30:00+08:00,2023-09-25 13:30:00+08:00,2023-09-26 13:30:00+08:00,2023-09-27 13:30:00+08:00,2023-09-28 13:30:00+08:00,2023-10-02 13:30:00+08:00,2023-10-03 13:30:00+08:00
starting_cash,10000000.0,10000000.0,2203905.142427,921903.876036,1851480.446535,1916878.693942,1959492.659321,1740449.09152
ending_cash,10000000.0,2203905.142427,921903.876036,1851480.446535,1916878.693942,1959492.659321,1740449.09152,1979378.550691
portfolio_value,10000000.0,9988642.942427,10009796.876036,9843482.746535,9799695.843942,9760817.859321,9642395.49152,9596488.900691
transactions,[],"[{'amount': 106951, 'dt': 2023-09-22 13:30:00+...","[{'amount': -37450, 'dt': 2023-09-25 13:30:00+...","[{'amount': -14450, 'dt': 2023-09-26 13:30:00+...","[{'amount': -87136, 'dt': 2023-09-27 13:30:00+...","[{'amount': -157495, 'dt': 2023-09-28 13:30:00...","[{'amount': -85400, 'dt': 2023-10-02 13:30:00+...","[{'amount': -3988, 'dt': 2023-10-03 13:30:00+0..."
shorts_count,0,0,0,0,0,0,0,0
positions,[],"[{'sid': Equity(34 [2886]), 'amount': 106951, ...","[{'sid': Equity(46 [4904]), 'amount': 14450, '...","[{'sid': Equity(25 [2609]), 'amount': 87136, '...","[{'sid': Equity(34 [2886]), 'amount': 104027, ...","[{'sid': Equity(34 [2886]), 'amount': 103976, ...","[{'sid': Equity(34 [2886]), 'amount': 103564, ...","[{'sid': Equity(34 [2886]), 'amount': 102579, ..."
short_value,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
short_exposure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
positions, transactions, orders = get_transaction_detail(stats)

In [19]:
transactions

,sid,symbol,amount,dt,price,order_id,asset,commission
2023-09-22 13:30:00+08:00,34,2886,106951,2023-09-22 13:30:00+08:00,37.800273,bdcb9758340444948f3af991c6b2478b,Equity(34 [2886]),None
2023-09-22 13:30:00+08:00,46,4904,51900,2023-09-22 13:30:00+08:00,72.104506,acc3c2d89f8e48c3bc42daa9e8d81560,Equity(46 [4904]),None
2023-09-25 13:30:00+08:00,46,4904,-37450,2023-09-25 13:30:00+08:00,72.895444,71a11f3296354834956e3a801721c806,Equity(46 [4904]),None
2023-09-25 13:30:00+08:00,34,2886,-106951,2023-09-25 13:30:00+08:00,37.999742,91e9060255b8480c88251ce5ed781b49,Equity(34 [2886]),None
2023-09-25 13:30:00+08:00,24,2603,34894,2023-09-25 13:30:00+08:00,116.000161,2777dbff1bbd48188466187cc31e69d3,Equity(24 [2603]),None
2023-09-25 13:30:00+08:00,6,1590,4064,2023-09-25 13:30:00+08:00,981.008970,1ec8dcf092f84220a87650c5db8959fc,Equity(6 [1590]),None
2023-09-26 13:30:00+08:00,46,4904,-14450,2023-09-26 13:30:00+08:00,72.699787,ceec5c24911044baba3d8c0abab96960,Equity(46 [4904]),None
2023-09-26 13:30:00+08:00,24,2603,-34894,2023-09-26 13:30:00+08:00,115.499884,dc7128121a9d40d182ec3d9d4d465492,Equity(24 [2603]),None
2023-09-26 13:30:00+08:00,6,1590,-4062,2023-09-26 13:30:00+08:00,957.984660,c55dc4e5611246698718518b5ad46860,Equity(6 [1590]),None
2023-09-26 13:30:00+08:00,25,2609,87136,2023-09-26 13:30:00+08:00,45.950388,5ca0608910f8443a9c2ec23de6c1ab6f,Equity(25 [2609]),None


In [20]:
positions['mv'] = positions['amount'] * positions['last_sale_price']
positions

,sid,symbol,asset,amount,cost_basis,last_sale_price,mv
2023-09-22 13:30:00+08:00,34,2886,Equity(34 [2886]),106951,37.854139,37.80,4042747.80
2023-09-22 13:30:00+08:00,46,4904,Equity(46 [4904]),51900,72.207262,72.10,3741990.00
2023-09-25 13:30:00+08:00,46,4904,Equity(46 [4904]),14450,73.043317,72.90,1053405.00
2023-09-25 13:30:00+08:00,24,2603,Equity(24 [2603]),34894,116.165462,116.00,4047704.00
2023-09-25 13:30:00+08:00,6,1590,Equity(6 [1590]),4064,982.407100,981.00,3986784.00
2023-09-26 13:30:00+08:00,25,2609,Equity(25 [2609]),87136,46.015872,45.95,4003899.20
2023-09-26 13:30:00+08:00,34,2886,Equity(34 [2886]),105366,37.904276,37.85,3988103.10
2023-09-27 13:30:00+08:00,34,2886,Equity(34 [2886]),104027,37.906429,37.70,3921817.90
2023-09-27 13:30:00+08:00,33,2885,Equity(33 [2885]),157495,25.186114,25.15,3960999.25
2023-09-28 13:30:00+08:00,34,2886,Equity(34 [2886]),103976,37.906679,37.70,3919895.20


In [21]:
stats.net_leverage

2023-09-21 13:30:00+08:00    0.000000
2023-09-22 13:30:00+08:00    0.779359
2023-09-25 13:30:00+08:00    0.907900
2023-09-26 13:30:00+08:00    0.811908
2023-09-27 13:30:00+08:00    0.804394
2023-09-28 13:30:00+08:00    0.799249
2023-10-02 13:30:00+08:00    0.819500
2023-10-03 13:30:00+08:00    0.793739
Name: net_leverage, dtype: float64

<span id="Case2"></span>
## Case 2 調整max_leverage
[Return to Menu](#menu)

接續**Case 1**，多調整`max_leverage=0.70`，其餘與**Case 1**相同。

In [22]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     max_leverage=0.70,
                     pipeline=make_pipeline,
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2024-03-13 02:41:05.749665]: INFO: rebalance: Cancel_order: current time: 2023-09-25 , created: 2023-09-25 , asset: Equity(46 [4904]), amount: -11093 , filled: 0
[2024-03-13 02:41:05.753066]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 47789.27943776
[2024-03-13 02:41:05.754062]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 3554, cost_basis: 982.73, last_sale_price: 981.0
[2024-03-13 02:41:05.754062]: INFO: handle_split: returning cash: 809.13
[2024-03-13 02:41:05.795164]: INFO: handle_simulation_end: Simulated 8 trading days
first open: 2023-09-21 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [23]:
stats.net_leverage

2023-09-21 13:30:00+08:00    0.000000
2023-09-22 13:30:00+08:00    0.704456
2023-09-25 13:30:00+08:00    0.783204
2023-09-26 13:30:00+08:00    0.708862
2023-09-27 13:30:00+08:00    0.703443
2023-09-28 13:30:00+08:00    0.698989
2023-10-02 13:30:00+08:00    0.715883
2023-10-03 13:30:00+08:00    0.694088
Name: net_leverage, dtype: float64

<span id="Case3"></span>
## Case 3 調整tradeday
[Return to Menu](#menu)  

接續**Case 1**，多新增`tradeday`，其餘與**Case 1**相同。

In [24]:
# 設定再平衡日期
freq = 'MS'   # QS-JUL  MS W
_tradeday = list(pd.date_range(start=start_dt, end=end_dt, freq=freq))
tradeday = [get_calendar(calendar_name).next_open(pd.Timestamp(i)).strftime('%Y-%m-%d') if \
           get_calendar(calendar_name).is_session(i)==False else i.strftime('%Y-%m-%d') for i in _tradeday]
tradeday

['2023-06-01', '2023-07-03', '2023-08-01', '2023-09-01', '2023-10-02']

In [25]:
algo_start_dt

Timestamp('2023-09-21 00:00:00+0000', tz='UTC')

In [26]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,           
                     tradeday=tradeday,
                     pipeline=make_pipeline,
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2024-03-13 02:41:05.883443]: INFO: handle_simulation_end: Simulated 8 trading days
first open: 2023-09-21 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [27]:
positions, transactions, orders = get_transaction_detail(stats)

In [28]:
transactions

,sid,symbol,amount,dt,price,order_id,asset,commission
2023-10-03 13:30:00+08:00,24,2603,35242,2023-10-03 13:30:00+08:00,110.500051,35a33e935bde4439b8e113256e10f083,Equity(24 [2603]),None
2023-10-03 13:30:00+08:00,34,2886,106382,2023-10-03 13:30:00+08:00,37.650179,fb0b44482bc54fff9b6dec3fd649e715,Equity(34 [2886]),None


<span id="Case4"></span>
## Case 4 調整rebalance_date_rule
[Return to Menu](#menu)  

接續**Case 1**，多新增`rebalance_date_rule`，並修改`start_session`為2023-09-01，其餘與**Case 1**相同。

In [29]:
from zipline.api import date_rules

algo = TargetPercentPipeAlgo(
                     start_session=pd.Timestamp('2023-09-01', tz='UTC'),
                     end_session=end_dt,
                     pipeline=make_pipeline,
                     # 每月的第四個交易日下單
                     rebalance_date_rule=date_rules.month_start(days_offset=3) 
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2024-03-13 02:41:06.016104]: INFO: handle_simulation_end: Simulated 22 trading days
first open: 2023-09-01 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [30]:
positions, transactions, orders = get_transaction_detail(stats)

In [31]:
orders

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-06 13:30:00+08:00,41,3034,4706e48bb8644098b53fd9a9c90ed314,2023-09-06 13:30:00+08:00,None,2023-09-06 13:30:00+08:00,9411,0,0,None,None,False,False,Equity(41 [3034]),0
2023-09-06 13:30:00+08:00,14,2327,ca20e3e3e6554702a701388075f7b1f3,2023-09-06 13:30:00+08:00,None,2023-09-06 13:30:00+08:00,7920,0,0,None,None,False,False,Equity(14 [2327]),0
2023-09-07 13:30:00+08:00,41,3034,4706e48bb8644098b53fd9a9c90ed314,2023-09-07 13:30:00+08:00,None,2023-09-06 13:30:00+08:00,9411,9411,5727,None,None,False,False,Equity(41 [3034]),1
2023-09-07 13:30:00+08:00,14,2327,ca20e3e3e6554702a701388075f7b1f3,2023-09-07 13:30:00+08:00,None,2023-09-06 13:30:00+08:00,7920,7920,5666,None,None,False,False,Equity(14 [2327]),1


<span id="Case5"></span>
## Case 5 調整slippage_model
[Return to Menu](#menu)  

接續**Case 1**，多新增`slippage_model`，將`volume_limit`由**0.025**調整為**0.01**，其餘與**Case 1**相同。

In [32]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     pipeline=make_pipeline,
                     slippage_model=slippage.VolumeShareSlippage(volume_limit=0.01, price_impact=0.1)
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2024-03-13 02:41:06.113628]: INFO: rebalance: Cancel_order: current time: 2023-09-22 , created: 2023-09-21 , asset: Equity(46 [4904]), amount: 55478 , filled: 20760
[2024-03-13 02:41:06.121367]: INFO: rebalance: Cancel_order: current time: 2023-09-25 , created: 2023-09-25 , asset: Equity(46 [4904]), amount: -5780 , filled: 0
[2024-03-13 02:41:06.121367]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54629.7584124
[2024-03-13 02:41:06.121367]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4063, cost_basis: 982.73, last_sale_price: 981.0
[2024-03-13 02:41:06.121367]: INFO: handle_split: returning cash: 643.62
[2024-03-13 02:41:06.132990]: INFO: rebalance: Cancel_order: current time: 2023-09-26 , created: 2023-09-26 , asset: Equity(6 [1590]), amount: -853 , filled: 0
[2024-03-13 02:41:06.138066]: INFO: rebalance: Cancel_order: current time: 2023-09-27 , created: 2023-09-26 , asset: Equity(33 [2885]), amount:

In [33]:
positions, transactions, orders = get_transaction_detail(stats)

In [34]:
orders.query('(symbol == "1590") & (created.dt.strftime("%Y-%m-%d") == "2023-09-25")')

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-25 13:30:00+08:00,6,1590,3217f240f39a4bc88fcdce896268d7cc,2023-09-25 13:30:00+08:00,None,2023-09-25 13:30:00+08:00,-4065,0,0.0,None,None,False,False,Equity(6 [1590]),0
2023-09-26 13:30:00+08:00,6,1590,3217f240f39a4bc88fcdce896268d7cc,2023-09-26 13:30:00+08:00,None,2023-09-25 13:30:00+08:00,-4063,-3210,13609.0,None,None,False,False,Equity(6 [1590]),2


In [35]:
# 321000 * 1% = 3210(股) 

df_bundle_price.query('(symbol == "1590") & (date.dt.strftime("%Y-%m-%d") == "2023-09-26")')[['symbol','date','volume']]

,symbol,date,volume
4486,1590,2023-09-26 00:00:00+00:00,321000.0


In [36]:
orders.query('(symbol == "2885") & (created.dt.strftime("%Y-%m-%d") == "2023-09-27")')

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-27 13:30:00+08:00,33,2885,06063291dd4e449a8871819852919f94,2023-09-27 13:30:00+08:00,None,2023-09-27 13:30:00+08:00,-151500,0,0.0,None,None,False,False,Equity(33 [2885]),0
2023-09-28 13:30:00+08:00,33,2885,06063291dd4e449a8871819852919f94,2023-09-28 13:30:00+08:00,None,2023-09-27 13:30:00+08:00,-151500,-107380,11904.0,None,None,False,False,Equity(33 [2885]),2


In [37]:
# 10738000 * 1% = 107380(股) 

df_bundle_price.query('(symbol == "2885") & (date.dt.strftime("%Y-%m-%d") == "2023-09-28")')[['symbol','date','volume']]

,symbol,date,volume
4625,2885,2023-09-28 00:00:00+00:00,10738000.0


<span id="Case6"></span>
## Case 6 調整stocklist
[Return to Menu](#menu)  


接續**Case 1**，多新增`stocklist`，其餘與**Case 1**相同。  



註1：`stocklist`限制是在pipeline執行完後。  

註2：也可以使用pipeline直接限制股票池。

In [38]:
len(StockList)

55

In [39]:
_StockList = [i for i in StockList if i!='2886']
len(_StockList)

54

In [40]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,            
                     stocklist=_StockList,
                     pipeline=make_pipeline,
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2024-03-13 02:41:06.339500]: INFO: rebalance: Cancel_order: current time: 2023-09-22 , created: 2023-09-21 , asset: Equity(46 [4904]), amount: 55478 , filled: 51900
[2024-03-13 02:41:06.346237]: INFO: rebalance: Cancel_order: current time: 2023-09-25 , created: 2023-09-25 , asset: Equity(46 [4904]), amount: -14450 , filled: 0
[2024-03-13 02:41:06.349619]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54643.19746736
[2024-03-13 02:41:06.350619]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4064, cost_basis: 982.73, last_sale_price: 981.0
[2024-03-13 02:41:06.350619]: INFO: handle_split: returning cash: 643.29
[2024-03-13 02:41:06.385375]: INFO: handle_simulation_end: Simulated 8 trading days
first open: 2023-09-21 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [41]:
positions, transactions, orders = get_transaction_detail(stats)

In [42]:
positions

,sid,symbol,asset,amount,cost_basis,last_sale_price
2023-09-22 13:30:00+08:00,46,4904,Equity(46 [4904]),51900,72.207262,72.10
2023-09-25 13:30:00+08:00,46,4904,Equity(46 [4904]),14450,73.043317,72.90
2023-09-25 13:30:00+08:00,24,2603,Equity(24 [2603]),34915,116.165477,116.00
2023-09-25 13:30:00+08:00,6,1590,Equity(6 [1590]),4066,982.407159,981.00
2023-09-26 13:30:00+08:00,25,2609,Equity(25 [2609]),87157,46.015867,45.95
2023-09-27 13:30:00+08:00,33,2885,Equity(33 [2885]),157624,25.186117,25.15
2023-09-28 13:30:00+08:00,25,2609,Equity(25 [2609]),85610,45.515095,45.45
2023-10-02 13:30:00+08:00,6,1590,Equity(6 [1590]),3997,1006.440770,1005.00
2023-10-03 13:30:00+08:00,24,2603,Equity(24 [2603]),34102,110.657517,110.50


<span id="Case7"></span>
## Case 7 調整order_filling_policy
[Return to Menu](#menu) 

接續**Case 1**，多新增`order_filling_policy='current_bar'`，其餘與**Case 1**相同。

In [43]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     pipeline=make_pipeline,
                     order_filling_policy='current_bar'
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2024-03-13 02:41:06.476428]: INFO: rebalance: Cancel_order: current time: 2023-09-25 , created: 2023-09-25 , asset: Equity(46 [4904]), amount: -3578 , filled: 0
[2024-03-13 02:41:06.523514]: INFO: handle_simulation_end: Simulated 8 trading days
first open: 2023-09-21 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [44]:
positions, transactions, orders = get_transaction_detail(stats)

In [45]:
result.loc['2023-09-22'].query('longs == True')

,longs
Equity(6 [1590]),True
Equity(24 [2603]),True


In [46]:
# 從`orders`中可以發現created=2023-09-22的單在當天就成交（status由0變為1）
orders.loc['2023-09-22']

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-22 13:30:00+08:00,34,2886,36e7f2eb3d37492e829558f5748560aa,2023-09-22 13:30:00+08:00,None,2023-09-22 08:45:00+08:00,-106951,-106951,17890.0,None,None,False,False,Equity(34 [2886]),1
2023-09-22 13:30:00+08:00,46,4904,0fc833a0c418408dae3da988454b2029,2023-09-22 13:30:00+08:00,None,2023-09-22 08:45:00+08:00,-55478,-51900,16559.0,None,None,False,False,Equity(46 [4904]),0
2023-09-22 13:30:00+08:00,24,2603,737aa5d17c9c43ff814b7c09a94898fe,2023-09-22 13:30:00+08:00,None,2023-09-22 08:45:00+08:00,35043,35043,5718.0,None,None,False,False,Equity(24 [2603]),1
2023-09-22 13:30:00+08:00,6,1590,50897cf4326c46abbea7efdf8869a5af,2023-09-22 13:30:00+08:00,None,2023-09-22 08:45:00+08:00,4081,4081,5717.0,None,None,False,False,Equity(6 [1590]),1


<span id="Case8"></span>
## Case 8 調整allow_short
[Return to Menu](#menu) 

接續**Case 1**，多新增`allow_short=True`，其餘與**Case 1**相同。  


以下設定pipeline（`make_pipeline()`），並定義`shorts`欄位用來判斷須放空的股票。在`shorts`欄位中要放空的股票標記為True，反之標記為False。

In [47]:
def make_pipeline():
    rsi = RSI()
    longs = rsi.top(2, mask = ~SingleAsset(bundle.asset_finder.lookup_symbol('IR0001', as_of_date=None)))
    shorts = rsi.bottom(2, mask = ~SingleAsset(bundle.asset_finder.lookup_symbol('IR0001', as_of_date=None)))

    return Pipeline(
        columns = {
            "longs" : longs,
            "shorts" : shorts
        }
    )

In [48]:
result = run_pipeline(make_pipeline(), algo_start, end)
result.query('(longs == True) | (shorts == True)' )

longs  shorts
2023-09-22 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(39 [2912])  False    True
2023-09-25 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(25 [2609])   True   False
                          Equity(31 [2883])  False    True
                          Equity(34 [2886])   True   False
2023-09-26 00:00:00+00:00 Equity(8 [2002])   False    True
                          Equity(10 [2301])  False    True
                          Equity(33 [2885])   True   False
                          Equity(34 [2886])   True   False
2023-09-27 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(17 [2357])  False    True
                          Equity(25 [2609])   True   False
                          Equity(34 [2886])   True   False
2023-09-28 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-02 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-03 00:00:00+00:00 Equity(2 [1301])   False    True
                          Equity(6 [1590])    True   False
                          Equity(14 [2327])   True   False
                          Equity(30 [2882])  False    True

In [49]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     allow_short=True,
                     pipeline=make_pipeline,
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2024-03-13 02:41:06.669320]: INFO: rebalance: Cancel_order: current time: 2023-09-22 , created: 2023-09-21 , asset: Equity(46 [4904]), amount: 55478 , filled: 51900
[2024-03-13 02:41:06.681121]: INFO: rebalance: Cancel_order: current time: 2023-09-25 , created: 2023-09-25 , asset: Equity(46 [4904]), amount: -14450 , filled: 0
[2024-03-13 02:41:06.687058]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54414.73353304
[2024-03-13 02:41:06.687453]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4047, cost_basis: 982.73, last_sale_price: 981.0
[2024-03-13 02:41:06.687453]: INFO: handle_split: returning cash: 648.82
[2024-03-13 02:41:06.734225]: INFO: rebalance: Cancel_order: current time: 2023-10-02 , created: 2023-09-28 , asset: Equity(39 [2912]), amount: -14669 , filled: -8575
[2024-03-13 02:41:06.747551]: INFO: handle_simulation_end: Simulated 8 trading days
first open: 2023-09-21 01:01:00+00:00
last close: 

In [50]:
positions, transactions, orders = get_transaction_detail(stats)

In [51]:
# 當天取消
orders.query('(symbol == "2912") & (created.dt.strftime("%Y-%m-%d") == "2023-09-28")')

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-28 13:30:00+08:00,39,2912,2ed71c04d41546fabc89536cab7b49de,2023-09-28 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,-14669,0,0.0,None,None,False,False,Equity(39 [2912]),0
2023-10-02 13:30:00+08:00,39,2912,2ed71c04d41546fabc89536cab7b49de,2023-10-02 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,-14669,-8575,9980.0,None,None,False,False,Equity(39 [2912]),2


In [52]:
# 343000 * 2.5% = 8575(股) 

df_bundle_price.query('(symbol == "2912") & (date.dt.strftime("%Y-%m-%d") == "2023-10-02")')[['symbol','date','volume']]

,symbol,date,volume
4687,2912,2023-10-02 00:00:00+00:00,343000.0


In [53]:
positions['mv'] = positions['amount'] * positions['last_sale_price']
positions.query('(symbol == "2912")')

,sid,symbol,asset,amount,cost_basis,last_sale_price,mv
2023-09-25 13:30:00+08:00,39,2912,Equity(39 [2912]),-15079,263.315631,264.5,-3988395.5
2023-10-02 13:30:00+08:00,39,2912,Equity(39 [2912]),-8575,261.819714,263.0,-2255225.0
2023-10-03 13:30:00+08:00,39,2912,Equity(39 [2912]),-14152,261.433302,262.0,-3707824.0


<span id="Case9"></span>
## Case 9 調整cancel_datedelta
[Return to Menu](#menu) 

接續**Case 8**，多新增`cancel_datedelta=2`，其餘與**Case 8**相同。  

In [54]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     allow_short=True,
                     cancel_datedelta=2,
                     pipeline=make_pipeline,
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2024-03-13 02:41:06.883576]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54414.73353304
[2024-03-13 02:41:06.883576]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4047, cost_basis: 982.73, last_sale_price: 981.0
[2024-03-13 02:41:06.883576]: INFO: handle_split: returning cash: 648.82
[2024-03-13 02:41:06.951237]: INFO: handle_simulation_end: Simulated 8 trading days
first open: 2023-09-21 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [55]:
result = run_pipeline(make_pipeline(), algo_start, end)
result.query('(longs == True) | (shorts == True)' )

longs  shorts
2023-09-22 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(39 [2912])  False    True
2023-09-25 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(25 [2609])   True   False
                          Equity(31 [2883])  False    True
                          Equity(34 [2886])   True   False
2023-09-26 00:00:00+00:00 Equity(8 [2002])   False    True
                          Equity(10 [2301])  False    True
                          Equity(33 [2885])   True   False
                          Equity(34 [2886])   True   False
2023-09-27 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(17 [2357])  False    True
                          Equity(25 [2609])   True   False
                          Equity(34 [2886])   True   False
2023-09-28 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-02 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-03 00:00:00+00:00 Equity(2 [1301])   False    True
                          Equity(6 [1590])    True   False
                          Equity(14 [2327])   True   False
                          Equity(30 [2882])  False    True

In [56]:
positions, transactions, orders = get_transaction_detail(stats)

In [57]:
orders.query('(symbol == "2912") & (created.dt.strftime("%Y-%m-%d") == "2023-09-28")')

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-28 13:30:00+08:00,39,2912,05ab5284f0904f0599fce02cafe7dd93,2023-09-28 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,-14660,0,0.0,None,None,False,False,Equity(39 [2912]),0
2023-10-02 13:30:00+08:00,39,2912,05ab5284f0904f0599fce02cafe7dd93,2023-10-02 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,-14660,-8575,9980.0,None,None,False,False,Equity(39 [2912]),0
2023-10-03 13:30:00+08:00,39,2912,05ab5284f0904f0599fce02cafe7dd93,2023-10-03 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,-14660,-14660,17035.0,None,None,False,False,Equity(39 [2912]),1


In [58]:
# 10/02：343000 * 2.5% = 8575(股) 
# 10/03：808000 * 2.5% = 20200(股) 

df_bundle_price.query('(symbol == "2912") & (date.dt.strftime("%Y-%m-%d")>="2023-10-02")')\
                 [['symbol','date','volume']]

,symbol,date,volume
4687,2912,2023-10-02 00:00:00+00:00,343000.0
4743,2912,2023-10-03 00:00:00+00:00,808000.0


In [59]:
positions['mv'] = positions['amount'] * positions['last_sale_price']
positions.query('(symbol == "2912")')

,sid,symbol,asset,amount,cost_basis,last_sale_price,mv
2023-09-25 13:30:00+08:00,39,2912,Equity(39 [2912]),-15079,263.315631,264.5,-3988395.5
2023-10-02 13:30:00+08:00,39,2912,Equity(39 [2912]),-8575,261.819714,263.0,-2255225.0
2023-10-03 13:30:00+08:00,39,2912,Equity(39 [2912]),-14660,261.412688,262.0,-3840920.0


<span id="Case10"></span>
## Case 10 調整limit_buy_multiplier
[Return to Menu](#menu) 

接續**Case 9**，多設定`limit_buy_multiplier=1.015`，其餘與**Case 9**相同。

In [60]:
result = run_pipeline(make_pipeline(), algo_start, end)
result.query('(longs == True) | (shorts == True)' )

longs  shorts
2023-09-22 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(39 [2912])  False    True
2023-09-25 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(25 [2609])   True   False
                          Equity(31 [2883])  False    True
                          Equity(34 [2886])   True   False
2023-09-26 00:00:00+00:00 Equity(8 [2002])   False    True
                          Equity(10 [2301])  False    True
                          Equity(33 [2885])   True   False
                          Equity(34 [2886])   True   False
2023-09-27 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(17 [2357])  False    True
                          Equity(25 [2609])   True   False
                          Equity(34 [2886])   True   False
2023-09-28 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-02 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-03 00:00:00+00:00 Equity(2 [1301])   False    True
                          Equity(6 [1590])    True   False
                          Equity(14 [2327])   True   False
                          Equity(30 [2882])  False    True

In [61]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     limit_buy_multiplier=1.015,
                     allow_short=True,
                     cancel_datedelta=2,
                     pipeline=make_pipeline,
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2024-03-13 02:41:07.183718]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54414.73353304
[2024-03-13 02:41:07.183718]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4047, cost_basis: 982.73, last_sale_price: 981.0
[2024-03-13 02:41:07.183718]: INFO: handle_split: returning cash: 648.82
[2024-03-13 02:41:07.254114]: INFO: exec_cancel: Cancel_order: current time: 2023-10-03,
                              due to : created>=current time + cancel_datedelta(2 days),
                              created: 2023-09-28, asset: Equity(6 [1590]), amount: 3931, filled: 0'
[2024-03-13 02:41:07.259895]: INFO: handle_simulation_end: Simulated 8 trading days
first open: 2023-09-21 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [62]:
positions, transactions, orders = get_transaction_detail(stats)

In [63]:
orders

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-21 13:30:00+08:00,34,2886,5e4ce9d6633d408a9389b175c652626b,2023-09-21 13:30:00+08:00,None,2023-09-21 13:30:00+08:00,106951,0,0.0,None,37.96,False,False,Equity(34 [2886]),0
2023-09-21 13:30:00+08:00,46,4904,e0b2c8d022ff41959b6f077bccad0c43,2023-09-21 13:30:00+08:00,None,2023-09-21 13:30:00+08:00,55478,0,0.0,None,73.18,False,False,Equity(46 [4904]),0
2023-09-21 13:30:00+08:00,10,2301,25b053b3403b4ec4aa3408745f592ac4,2023-09-21 13:30:00+08:00,None,2023-09-21 13:30:00+08:00,-33613,0,0.0,None,NaN,False,False,Equity(10 [2301]),0
2023-09-21 13:30:00+08:00,44,3231,7a065d9e8d604f699ec3efd6ca41d47c,2023-09-21 13:30:00+08:00,None,2023-09-21 13:30:00+08:00,-39800,0,0.0,None,NaN,False,False,Equity(44 [3231]),0
2023-09-22 13:30:00+08:00,34,2886,5e4ce9d6633d408a9389b175c652626b,2023-09-22 13:30:00+08:00,None,2023-09-21 13:30:00+08:00,106951,106951,5761.0,None,37.96,False,True,Equity(34 [2886]),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-03 13:30:00+08:00,39,2912,aaa092b0d9954807a887fab5edbed96e,2023-10-03 13:30:00+08:00,None,2023-10-03 13:30:00+08:00,14660,0,0.0,None,NaN,False,False,Equity(39 [2912]),0
2023-10-03 13:30:00+08:00,6,1590,01f05fe434e643268c1562ae1004734a,2023-10-03 13:30:00+08:00,None,2023-10-03 13:30:00+08:00,3762,0,0.0,None,1012.97,False,False,Equity(6 [1590]),0
2023-10-03 13:30:00+08:00,14,2327,606f2de8c38d4be484df5d6889fe5e28,2023-10-03 13:30:00+08:00,None,2023-10-03 13:30:00+08:00,7221,0,0.0,None,527.80,False,False,Equity(14 [2327]),0
2023-10-03 13:30:00+08:00,2,1301,d5aa6f02906f4e008bceac04ade72d21,2023-10-03 13:30:00+08:00,None,2023-10-03 13:30:00+08:00,-47471,0,0.0,None,NaN,False,False,Equity(2 [1301]),0


In [64]:
orders.query('(symbol == "1590") & (created.dt.strftime("%Y-%m-%d") == "2023-09-28")')

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-28 13:30:00+08:00,6,1590,7a1633e9d2314561bf5fc6b235784201,2023-09-28 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,3931,0,0.0,None,993.68,False,False,Equity(6 [1590]),0
2023-10-03 13:30:00+08:00,6,1590,7a1633e9d2314561bf5fc6b235784201,2023-10-03 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,3931,0,0.0,None,993.68,False,False,Equity(6 [1590]),2


In [65]:
# 9/28 979 * 1.015 = 993.685
df_bundle_price.query('(symbol == "1590") & (date.dt.strftime("%Y-%m-%d")>="2023-09-28")')\
                 [['symbol','date','close']]

,symbol,date,close
4598,1590,2023-09-28 00:00:00+00:00,979.0
4654,1590,2023-10-02 00:00:00+00:00,1005.0
4710,1590,2023-10-03 00:00:00+00:00,998.0


<span id="Case11"></span>
## Case 11 調整custom_weight、analyze、record_vars、get_record_vars與get_transaction_detail
[Return to Menu](#menu) 

接續**Case 10**，多設定`custom_weight`=True、`analyze`、`record_vars`、`get_record_vars`=True與`get_transaction_detail`=True，其餘與**Case 10**相同。

In [66]:
class Weight(CustomFactor):
    
    inputs =  [TQAltDataSet.Market_Cap_Dollars] 
    outputs = ["Market_Cap_Dollars","Sum_Market_Cap_Dollars","Weight"]   
    window_length = 1

    def compute(self, today, assets, out, Market_Cap_Dollars):
        
        out.Market_Cap_Dollars[:] = Market_Cap_Dollars
        out.Sum_Market_Cap_Dollars[:] = np.nansum(Market_Cap_Dollars, axis=1)
        out.Weight[:] = Market_Cap_Dollars / np.sum(Market_Cap_Dollars, axis=1)

In [67]:
def make_pipeline():
    
    rsi = RSI()
    longs = rsi.top(2, mask = ~SingleAsset(bundle.asset_finder.lookup_symbol('IR0001', as_of_date=None)))
    shorts = rsi.bottom(2, mask = ~SingleAsset(bundle.asset_finder.lookup_symbol('IR0001', as_of_date=None)))

    return Pipeline(
        
        columns = {
            "Market_Cap_Dollars":Weight().Market_Cap_Dollars,        
            "longs" : longs,
            "shorts" : shorts,
            "long_weights" : Weight(mask=longs).Weight,
            "short_weights" : Weight(mask=shorts).Weight
        }
    )

In [68]:
def analyze(context, perf):
    
    fig = plt.figure(figsize=(16, 24), dpi=400)
    
    # First chart(累積報酬)
    ax = fig.add_subplot(611) 
    ax.set_title('Strategy Results') 
    ax.plot(perf['algorithm_period_return'],
            linestyle='-', 
            label='algorithm period return',
            linewidth=3.0)
    ax.plot(perf['benchmark_period_return'],
            linestyle='-', 
            label='benchmark period return',
            linewidth=3.0)
    ax.legend()
    ax.grid(False)
    
    # Second chart(returns)
    ax = fig.add_subplot(612, sharex=ax)       
    ax.plot(perf['returns'],
            linestyle='-', 
            label='returns',
            linewidth=3.0)
    ax.legend()
    ax.grid(False)

    # Third chart(ending_cash) -> 觀察是否超買
    ax = fig.add_subplot(613, sharex=ax)
    ax.plot(perf['ending_cash'], 
            label='ending_cash',
            linestyle='-',
            linewidth=3.0)
    ax.axhline(y=1, c='r', linewidth=1)
    ax.legend()
    ax.grid(False)

    # Forth chart(shorts_count) -> 觀察是否放空
    ax = fig.add_subplot(614, sharex=ax)
    ax.plot(perf['shorts_count'], 
            label='shorts_count',
            linestyle='-',
            linewidth=3.0)
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.axhline(y=0, c='r', linewidth=1)
    ax.legend()
    ax.grid(False)
    
    # Fifth chart(longs_count)
    ax = fig.add_subplot(615, sharex=ax)
    ax.plot(perf['longs_count'], 
            label='longs_count',
            linestyle='-',
            linewidth=3.0)
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.axhline(y=0, c='r', linewidth=1)
    ax.legend()
    ax.grid(False) 
    
    # Sixth chart(weights) -> 觀察每日持股權重
    ax = fig.add_subplot(616, sharex=ax)        
    weights = pd.concat([df.to_frame(d) for d, df in perf['daily_weights'].dropna().items()],axis=1).T
    
    for i in weights.columns:
        df = weights.loc[:,i]
        ax.scatter(df.index,df.values,marker='.', s=5, c='grey', label='daily_weights')
    ax.axhline(y=0, c='r', linewidth=1)
    ax.legend(['daily_weights'])
    ax.grid(False)

    fig.tight_layout()

def record_vars(context, data):
    """
    Plot variables at the end of each day.
    """
            
    record(daily_weights=context.daily_weights,
           Market_Cap_Dollars=context.output.Market_Cap_Dollars
          )

In [69]:
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     limit_buy_multiplier=1.015,
                     allow_short=True,
                     custom_weight=True,
                     cancel_datedelta=2,
                     pipeline=make_pipeline,
                     analyze=analyze,
                     record_vars=record_vars,
                     get_record_vars=True,
                     get_transaction_detail=True
)

# set_algo_instance
set_algo_instance(algo)

# run
stats = algo.run()

[2024-03-13 02:41:07.483598]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 48703.13517504
[2024-03-13 02:41:07.483598]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 3622, cost_basis: 982.73, last_sale_price: 981.0
[2024-03-13 02:41:07.483598]: INFO: handle_split: returning cash: 787.02
[2024-03-13 02:41:07.583608]: INFO: exec_cancel: Cancel_order: current time: 2023-10-03,
                              due to : created>=current time + cancel_datedelta(2 days),
                              created: 2023-09-28, asset: Equity(6 [1590]), amount: 2145, filled: 0'
[2024-03-13 02:41:07.600275]: INFO: handle_simulation_end: Simulated 8 trading days
first open: 2023-09-21 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [70]:
result = run_pipeline(make_pipeline(), algo_start, end)
result.query('(longs == True) | (shorts == True)' )

Market_Cap_Dollars  longs   
2023-09-22 00:00:00+00:00 Equity(6 [1590])         1.980000e+11   True  \
                          Equity(10 [2301])        2.818339e+11  False   
                          Equity(24 [2603])        2.444465e+11   True   
                          Equity(39 [2912])        2.739405e+11  False   
2023-09-25 00:00:00+00:00 Equity(10 [2301])        2.842023e+11  False   
                          Equity(25 [2609])        1.599384e+11   True   
                          Equity(31 [2883])        1.970197e+11  False   
                          Equity(34 [2886])        5.311406e+11   True   
2023-09-26 00:00:00+00:00 Equity(8 [2002])         4.122534e+11  False   
                          Equity(10 [2301])        2.842023e+11  False   
                          Equity(33 [2885])        3.197649e+11   True   
                          Equity(34 [2886])        5.339508e+11   True   
2023-09-27 00:00:00+00:00 Equity(10 [2301])        2.842023e+11  False   
                          Equity(17 [2357])        2.659082e+11  False   
                          Equity(25 [2609])        1.604622e+11   True   
                          Equity(34 [2886])        5.318431e+11   True   
2023-09-28 00:00:00+00:00 Equity(6 [1590])         1.982000e+11   True   
                          Equity(10 [2301])        2.842023e+11  False   
                          Equity(34 [2886])        5.297354e+11   True   
                          Equity(39 [2912])        2.734207e+11  False   
2023-10-02 00:00:00+00:00 Equity(10 [2301])        2.877548e+11  False   
                          Equity(24 [2603])        2.455047e+11   True   
                          Equity(34 [2886])        5.297354e+11   True   
                          Equity(39 [2912])        2.729008e+11  False   
2023-10-03 00:00:00+00:00 Equity(2 [1301])         5.067130e+11  False   
                          Equity(6 [1590])         2.010000e+11   True   
                          Equity(14 [2327])        2.255366e+11   True   
                          Equity(30 [2882])        6.527799e+11  False   

                                             shorts  long_weights   
2023-09-22 00:00:00+00:00 Equity(6 [1590])    False      0.447512  \
                          Equity(10 [2301])    True           NaN   
                          Equity(24 [2603])   False      0.552488   
                          Equity(39 [2912])    True           NaN   
2023-09-25 00:00:00+00:00 Equity(10 [2301])    True           NaN   
                          Equity(25 [2609])   False      0.231433   
                          Equity(31 [2883])    True           NaN   
                          Equity(34 [2886])   False      0.768567   
2023-09-26 00:00:00+00:00 Equity(8 [2002])     True           NaN   
                          Equity(10 [2301])    True           NaN   
                          Equity(33 [2885])   False      0.374557   
                          Equity(34 [2886])   False      0.625443   
2023-09-27 00:00:00+00:00 Equity(10 [2301])    True           NaN   
                          Equity(17 [2357])    True           NaN   
                          Equity(25 [2609])   False      0.231780   
                          Equity(34 [2886])   False      0.768220   
2023-09-28 00:00:00+00:00 Equity(6 [1590])    False      0.272277   
                          Equity(10 [2301])    True           NaN   
                          Equity(34 [2886])   False      0.727723   
                          Equity(39 [2912])    True           NaN   
2023-10-02 00:00:00+00:00 Equity(10 [2301])    True           NaN   
                          Equity(24 [2603])   False      0.316682   
                          Equity(34 [2886])   False      0.683318   
                          Equity(39 [2912])    True           NaN   
2023-10-03 00:00:00+00:00 Equity(2 [1301])     True           NaN   
                          Equity(6 [1590])    False      0.471237   
                          Equity(14

#### algo.positions

In [71]:
# 計算實際股票部位的weight = 個股持股市值／所有股票部位的持股市值加總
pos = algo.positions

pos['mv'] = pos['amount'] * pos['last_sale_price']

positive_sum = pos[pos['mv'] > 0].groupby(level=0)['mv'].sum()
negative_sum = - pos[pos['mv'] < 0].groupby(level=0)['mv'].sum()
pos['sum'] = np.where(pos['mv'] > 0,
                            pos.index.map(positive_sum),
                            pos.index.map(negative_sum))

pos['weight'] = pos['mv'] / pos['sum']
pos

,sid,symbol,asset,amount,cost_basis,last_sale_price,mv,sum,weight
2023-09-22 13:30:00+08:00,34,2886,Equity(34 [2886]),147424,37.854390,37.80,5572627.20,8058923.60,0.691485
2023-09-22 13:30:00+08:00,46,4904,Equity(46 [4904]),34484,72.204762,72.10,2486296.40,8058923.60,0.308515
2023-09-22 13:30:00+08:00,10,2301,Equity(10 [2301]),-33495,119.468922,120.00,-4019400.00,8113352.50,-0.495406
2023-09-22 13:30:00+08:00,44,3231,Equity(44 [3231]),-39941,102.046428,102.50,-4093952.50,8113352.50,-0.504594
2023-09-25 13:30:00+08:00,10,2301,Equity(10 [2301]),-33646,119.468898,120.00,-4037520.00,7969312.50,-0.506633
2023-09-25 13:30:00+08:00,24,2603,Equity(24 [2603]),38418,116.165508,116.00,4456488.00,8011632.00,0.556252
2023-09-25 13:30:00+08:00,6,1590,Equity(6 [1590]),3624,982.405312,981.00,3555144.00,8011632.00,0.443748
2023-09-25 13:30:00+08:00,39,2912,Equity(39 [2912]),-14865,263.316060,264.50,-3931792.50,7969312.50,-0.493367
2023-09-26 13:30:00+08:00,10,2301,Equity(10 [2301]),-39051,119.468888,120.00,-4686120.00,7921019.20,-0.591606
2023-09-26 13:30:00+08:00,25,2609,Equity(25 [2609]),39964,46.015565,45.95,1836345.80,7910665.20,0.232135


#### algo.dict_record_vars

In [72]:
record_vars = algo.dict_record_vars

In [73]:
# 實際持股市值 = 個股持股市值／所有股票部位的持股市值加總 * max_leverage
record_vars['daily_weights']

,date,symbol,daily_weights
0,2023-09-22 00:00:00+08:00,2886,0.559924
1,2023-09-22 00:00:00+08:00,4904,0.249817
2,2023-09-22 00:00:00+08:00,2301,-0.403860
3,2023-09-22 00:00:00+08:00,3231,-0.411351
4,2023-09-25 00:00:00+08:00,2301,-0.407072
5,2023-09-25 00:00:00+08:00,2603,0.449313
6,2023-09-25 00:00:00+08:00,1590,0.358437
7,2023-09-25 00:00:00+08:00,2912,-0.396412
8,2023-09-26 00:00:00+08:00,2886,0.622353
9,2023-09-26 00:00:00+08:00,2301,-0.480123


In [74]:
# 個股總市值
record_vars['Market_Cap_Dollars']

,date,symbol,Market_Cap_Dollars
0,2023-09-21 00:00:00+08:00,1101,2.458148e+11
1,2023-09-21 00:00:00+08:00,1216,4.028549e+11
2,2023-09-21 00:00:00+08:00,1301,5.245370e+11
3,2023-09-21 00:00:00+08:00,1303,5.424682e+11
4,2023-09-21 00:00:00+08:00,1326,3.798049e+11
...,...,...,...
443,2023-10-03 00:00:00+08:00,6415,1.188855e+11
444,2023-10-03 00:00:00+08:00,6505,7.649346e+11
445,2023-10-03 00:00:00+08:00,6669,2.867389e+11
446,2023-10-03 00:00:00+08:00,9910,1.920654e+11


[Return to Menu](#menu) 